In [2]:
from utils.file_utils import get_config
from utils.dataset_utils import get_dataloader
from utils.training_utils import parse_data, get_scene_vehicle_nums
import torch

In [3]:
config = get_config('./config.yaml')
train_loader, val_loader, test_loader = get_dataloader(config)
gpu_id = torch.cuda.current_device()
for batch in train_loader:
    ## if no vehicles in the batch, skip it
    vehicle_nums_dic = get_scene_vehicle_nums(batch)
    if vehicle_nums_dic['cur'] == 0:
        print('No vehicles in the batch')
        continue
    input_dict, ground_truth_dict = parse_data(data=batch, gpu_id=gpu_id, config=config)
    break
    

In [4]:
from modules.SwinTransformerEncoder import SwinTransformerEncoder
swin = SwinTransformerEncoder(config=config)

ModuleNotFoundError: No module named 'core'